In [1]:
import pandas as pd
from geopandas import GeoDataFrame, GeoSeries
from rtree import index
from shapely.geometry import Point
from shapely.wkt import loads
from sqlalchemy import create_engine
from datetime import datetime

import re

class GeoDataFrame(GeoDataFrame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.spatial_index = index.Index()
    
    def build_rtree(self):
        for feature in self.iterfeatures(show_bbox=True):
            self.spatial_index.insert(
                    int(feature['id']),
                    (feature['bbox'][0],feature['bbox'][1],feature['bbox'][2],feature['bbox'][3])
            )
    
    def within_distance_of(self,match_features,dist=40, match_column='matched'):
        for feature in self.iterfeatures(show_bbox=True):
            matched = []
            
            for match in match_features.spatial_index.intersection(
                    (feature['bbox'][0]-dist,feature['bbox'][1]-dist,feature['bbox'][2]+dist,feature['bbox'][3]+dist)):
                if self.loc[int(feature['id']),'geometry'].distance(match_features.loc[int(match),'geometry']) < dist:
                    matched.append(match)
            
            self.loc[int(feature['id']),match_column] = str(matched)
            
    
# Set up Database connection for loading stop path geometries
ENGINE = create_engine('')


In [2]:
# Load APC Data for service day

def parse_date(date,time):
    date_str = '{0} {1}'.format(date, time)
    return datetime.strptime(date_str, '%m%d%Y %H%M%S')
    
May11 = pd.read_csv(
    r'\Kamloops\data_appl\apc\20160510\9875.dat',
    header=None, 
    names=[
        'Event',
        'NA',
        'APCUnitID',
        'FrontDoor',
        'RearDoor',
        'NA2','NA3',
        'FrontOn',
        'RearOn1',
        'RearOn2',
        'NA4',
        'FrontOff',
        'RearOff1',
        'RearOff2',
        'NA5','NA6',
        'RecordID',
        'Latitude',
        'Longitude',
        'Date',
        'Time'], 
    dtype=str,
    parse_dates={'DateTime':[-2,-1]},
    date_parser=parse_date)

May12 = pd.read_csv(
    r'\Kamloops\data_appl\apc\20160511\9875.dat',
    header=None, 
    names=[
        'Event',
        'NA',
        'APCUnitID',
        'FrontDoor',
        'RearDoor',
        'NA2','NA3',
        'FrontOn',
        'RearOn1',
        'RearOn2',
        'NA4',
        'FrontOff',
        'RearOff1',
        'RearOff2',
        'NA5','NA6',
        'RecordID',
        'Latitude',
        'Longitude',
        'Date',
        'Time'], 
    dtype=str,
    parse_dates={'DateTime':[-2,-1]},
    date_parser=parse_date)

df = pd.concat([May11,May12])
mask = (df['DateTime'] > '2016-5-10 04:00:00') & (df['DateTime'] <= '2016-5-11 04:00:00')
service_day = df.loc[mask].copy()

service_day['geometry'] = service_day.apply(lambda x: Point(float(x.Longitude), float(x.Latitude)), axis=1)
service_day = GeoDataFrame(service_day)
service_day.crs = {'init': 'epsg:4326', 'no_defs': True}
service_day.to_crs(epsg=3005, inplace=True)
# Set the index of APC data to datetime so we can query for records within certain time periods 
service_day.build_rtree()

In [3]:
stops = pd.read_sql('''
        SELECT
            stopid,
            SDO_UTIL.TO_WKTGEOMETRY(geom) as geometry
        FROM 
            src_bct_stop
        ''',
        ENGINE)

# Convert route geometry to python
stops['geometry'] = stops.apply(lambda x: loads(x.geometry), axis=1)
stops = GeoDataFrame(stops)
stops.crs = {'init': 'epsg:3005', 'no_defs': True}
stops.build_rtree()

In [4]:
routes = pd.read_sql('''
        SELECT
            route,
            SDO_UTIL.TO_WKTGEOMETRY(geom) as geometry
        FROM 
            src_bct_route
        WHERE 
            system is not NULL AND
            route is not NULL AND
            pattern is not NULL
        ''',
        ENGINE)

# Convert route geometry to python
routes['geometry'] = routes.apply(lambda x: loads(x.geometry), axis=1)
routes = GeoDataFrame(routes)
routes.crs = {'init': 'epsg:3005', 'no_defs': True}
routes['wgs'] = routes.to_crs(epsg=4326).geometry
routes.build_rtree()

In [ ]:
service_day.within_distance_of(routes, dist=40, match_column='matched_routes')
service_day.within_distance_of(stops, dist=40, match_column='matched_stops')

In [ ]:
routes['id'] = routes.index.values
stops['id'] = stops.index.values

def parse_matched_routes(row):
    df = pd.DataFrame(re.sub('\[|\]', "", row.matched_routes).split(","),columns=['id'])
    df['id'] = pd.to_numeric(df['id'])
    potential_routes = pd.merge(df,routes, how='left', left_on='id', right_on='id')
    return str(set(potential_routes['route'].values.tolist()))

def parse_matched_stops(row):
    df = pd.DataFrame(re.sub('\[|\]', "", row.matched_stops).split(","),columns=['id'])
    df['id'] = pd.to_numeric(df['id'])
    potential_stops = pd.merge(df,stops, how='left', left_on='id', right_on='id')
    return str(set(potential_stops['stopid'].values.tolist()))

service_day['matched_routes'] = service_day.apply(parse_matched_routes, axis=1)
service_day['matched_stops'] = service_day.apply(parse_matched_stops, axis=1)
service_day.to_csv('testing.csv')
service_day
